### arquivo .csv

In [1]:
# Import PySpark
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Create SparkSession
spark=SparkSession.builder.master("local[1]")\
        .appName("Music")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mssql-jdbc-7.4.1.jre8.jar" ) \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

txt = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/data.csv')


23/08/15 21:48:03 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
23/08/15 21:48:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 21:48:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.4.0
PySpark Version :3.4.0


In [1]:
class dados():
    def __init__(self,music,genres,artists,playlists,tracks):
        self.__txt01  = music
        self.__txt02  = genres
        self.__txt03  = artists
        self.__txt04  = playlists
        self.__txt05  = tracks


    def music(self):
        df = self.__txt01.select(col("id")
                ,col("acousticness").cast(DecimalType(3,2))
                ,regexp_replace(regexp_replace(regexp_replace(col("artists"),r"\[",""),r"\]",""),"'","").alias("artists")
                ,col("danceability").cast(DecimalType(3,2))
                ,col("duration_ms")
                ,col("energy").cast(DecimalType(3,2))
                ,col("explicit")
                ,col("instrumentalness").cast(DecimalType(3,2))
                ,col("key")
                ,col("liveness").cast(DecimalType(3,2))
                ,col("mode")
                ,col("loudness").cast(DecimalType(5,2))
                ,col("name")
                ,col("popularity")
                ,when(col("release_date")=="1928",lit("1928-01-01")).otherwise(col("release_date")).alias("release_date")
                ,col("speechiness").cast(DecimalType(5,2))
                ,col("tempo").cast(DecimalType(6,3))
                ,col("valence").cast(DecimalType(5,2))
                ,col("year")
                )
        df = df.withColumn("rowid", monotonically_increasing_id())
        return df
    
    def genre(self):
        dfm = dados.music(self)
        dfs = self.__txt02.select(col("id"),
                                 col("danceability").cast(DecimalType(3,2)),
                                 col("energy").cast(DecimalType(3,2)),
                                 col("key"),
                                 col("loudness").cast(DecimalType(4,2)),
                                 col("mode"),
                                 col("speechiness").cast(DecimalType(3,2)),
                                 col("acousticness").cast(DecimalType(3,2)),
                                 col("instrumentalness").cast(DecimalType(3,2)),
                                 col("liveness").cast(DecimalType(3,2)),
                                 col("valence").cast(DecimalType(3,2)),
                                 col("tempo").cast(DecimalType(3,2)),
                                 col("type"),
                                 col("uri"),
                                 col("track_href"),
                                 col("analysis_url"),
                                 col("duration_ms"),
                                 col("time_signature"),
                                 col("genre"),
                                 col("song_name"))
        dfs = dfs.withColumn("rowid", monotonically_increasing_id())
        return dfs
    
    def artists(self):
        dfs = self.__txt03.select( col("followers")
                                  ,regexp_replace("genres","[^a-zA-Z0-9]","").alias("genres")
                                  ,col("name")
                                  ,col("popularity")
                                )  
        dfs = dfs.withColumn("id", row_number().over(Window.partitionBy("name").orderBy("name")))
        dfs = dfs.select("id"
                         ,"genres"
                         ,regexp_replace("name",r"\"","").alias("name")
                         ,regexp_replace("popularity",r"\"","").alias("popularity")
                         ,"followers")
        
        dfs = dfs.withColumn("rowid", monotonically_increasing_id())\
                 .withColumn("teste", lit(1)+ (dfs["rowid"]-1))

    

        return dfs
    def playlists(self):
        dfs = self.__txt04.select("Playlist","Genre")
        dfs = dfs.withColumn("rowid", monotonically_increasing_id())

        return dfs
    
    def tracks(self):
        dfs = self.__txt05.select("id"
                                  ,"name"
                                  ,"popularity"
                                  ,"duration_ms"
                                  ,"explicit"
                                  ,regexp_replace("artists","[^a-zA-Z0-9]","").alias("artists")
                                  ,regexp_replace("id_artists","[^a-zA-Z0-9]","").alias("id_artists")
                                  )

        return dfs
    
    def Runjob(self):
        dfmusic  = self.music()
        dfgenre  = self.genre()
        dfartist = self.artists()
        
        dfgenre = dfgenre.drop("acousticness","danceability","duration_ms","energy","instrumentalness","key","liveness","loudness","mode","rowid","speechiness","tempo","valence")

        df = dfmusic.join(dfgenre,"id","left")\
                    .join(dfartist,["rowid","id","name","popularity"],"full")

        return df 

In [45]:
music = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/data.csv')
tracks = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/tracks.csv')
genres = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/genres_v2.csv')
artists = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/artists.csv')
playlists = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/playlists.csv')

x = dados(music,genres,artists,playlists,tracks)
df = x.artists()
df.show()     

odate = datetime.now().strftime("%Y-%m-%d")

print(odate)

# df.write.parquet(f"/Users/eduardoalberto/LoadFile/parquet/arqui01/{odate}/")


+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+-----+
|                  id|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|          type|                 uri|          track_href|        analysis_url|duration_ms|time_signature|    genre|           song_name|rowid|
+--------------------+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+-----+
|2Vc6NJ9PW9gD9q343...|        0.83|  0.81|  2|   -7.36|   1|       0.42|        0.06|            0.01|    0.06|   0.39| null|audio_features|spotify:track:2Vc...|https://api.spoti...|https://api.sp

In [ ]:
rom  jobMusic import *
import os.path as path

music       = "/Users/eduardoalberto/LoadFile/archive/data.csv"
tracks      = "/Users/eduardoalberto/LoadFile/archive/tracks.csv"
genres      = "/Users/eduardoalberto/LoadFile/archive/genres_v2.csv"
artists     = "/Users/eduardoalberto/LoadFile/archive/artists.csv"
playlists   = "/Users/eduardoalberto/LoadFile/archive/playlists.csv"

if path.exists(music):
    music = spark.read.option("header", "true").option('inferSchema', 'true').csv(music)
else:
    print(f"arquivo {music} ")

if path.exists(tracks):
    tracks = spark.read.option("header", "true").option('inferSchema', 'true').csv(tracks)
else:
    print(f"arquivo {tracks} ")

if path.exists(genres):
    genres = spark.read.option("header", "true").option('inferSchema', 'true').csv(genres)
else:
    print(f"arquivo {genres} ")

if path.exists(artists):
    artists = spark.read.option("header", "true").option('inferSchema', 'true').csv(artists)
else:
    print(f"arquivo {artists} ")
    
if path.exists(playlists):
    playlists = spark.read.option("header", "true").option('inferSchema', 'true').csv(playlists)
else:
    print(f"arquivo {playlists} ")
